##Setup

In [1]:
#connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U transformers
#!pip install vllm==0.5.3.post1
!pip install -U torch
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 66.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.1 MB/s eta 0:00:00


In [4]:

#get data file
!wget -q -O data.csv "https://docs.google.com/uc?export=download&id=13NuKWse5ZLbE7-3WtmdzYSeOOaJ3MydG"


In [5]:
import pandas as pd

data = pd.read_csv("data.csv")
#print(data.head())
print(data.columns)

Index(['id_1', 'id_2', 'text_1', 'text_2', '1_better', 'prompt'], dtype='object')


In [6]:
df = data.sample(n=100, random_state=42)
print(df.columns)

Index(['id_1', 'id_2', 'text_1', 'text_2', '1_better', 'prompt'], dtype='object')


In [7]:
rubric = '''A high-scoring essay has native-like facility in the use of language with syntactic variety, appropriate word choice and phrases; well-controlled text organization; precise use of grammar and conventions; rare language inaccuracies that do not impede communication.
A low scoring essay has a limited range of familiar words or phrases loosely strung together; frequent errors in grammar (including syntax) and usage. Communication impeded in most cases by language inaccuracies.'''

rubric_cohesion = '''A high-scoring essay has text organization consistently well-controlled using a variety of effective linguistic features such as reference and transitional words and phrases to connect ideas across sentences and paragraphs; appropriate overlap of ideas.
A low scoring essay has no clear control of organization; cohesive devices not present or unsuccessfully used; presentation of ideas unclear'''

rubric_syntax = '''A high-scoring essay has flexible and effective use of a full range of syntactic structures including simple, compound, and complex sentences; there may be rare minor and negligible errors in sentence formation.
A low-scoring essay has pervasive and basic errors in sentence structure and word order that cause confusion; basic sentence errors are common.'''


rubric_vocab = '''A high-scoring essay has a wide range of vocabulary flexibly and effectively used to convey precise meanings; skillful use of topic-related terms and less common words; rare negligible inaccuracies in word use.
A low scoring essay has limited vocabulary often inappropriately used; limited control of word choice and word forms; little attempt to use topic-related terms.'''


rubric_phrase = '''A high scoring essay has flexible and effective use of a variety of phrases, such as idioms, collocations, and lexical bundles, to convey precise and subtle meanings; rare minor inaccuracies that are negligible.
A low scoring essay hsa memorized chunks of language, or simple phrasal patterns, predominate; many repetitions and misuses of phrases. '''


rubric_grammar = '''A high scoring essay has command of grammar and usage with few or no errors. A low scoring essay has errors in grammar and usage throughout.'''


rubric_conv = '''A high scoring essay has consistent use of appropriate conventions to convey meaning; spelling, capitalization, and punctuation errors nonexistent or negligible. A low scoring essay has minimal use of conventions; spelling, capitalization, and punctuation errors throughout.'''


In [8]:
from datasets import Dataset
from torch.utils.data import DataLoader
from torch import cuda, tensor, no_grad
import torch
from tqdm.auto import tqdm
import json


def preprocess(data):
  batch = {}
  # Modified for Gemma format which uses 'role' and 'parts' instead of 'messages'
  batch['text'] = [f"<start_of_turn>system\nYou are an essay grader who grades an essay according to the following rubric: {rubric}<end_of_turn>\n<start_of_turn>user\nWhich of the following essays scores higher based on the rubric? Essay A: {essay_1}\n Essay B: {essay_2}\n Output which essay, A or B, scores HIGHER.<end_of_turn>\n<start_of_turn>model\n" for essay_1, essay_2 in zip(data['text_1'], data['text_2'])]

  batch['id_1'] = data['id_1']
  batch['id_2'] = data['id_2']
  batch['prompt'] = data['prompt']

  return batch

def compute_metric():
    return

def collate(data):
    return {'text': [ex['text'] for ex in data],
            'id_1': [ex['id_1'] for ex in data],
            'id_2': [ex['id_2'] for ex in data],
            'prompt': [ex['prompt'] for ex in data]}

batch_size = 8
print('loading dataset...\n')
train_data = Dataset.from_pandas(df)
# train_data = train_data.select_columns(['full_text', 'text_id_kaggle'])
train_data = train_data.map(preprocess, batched=True, batch_size=batch_size)
train_dataloader = DataLoader(train_data,
                               batch_size=batch_size, shuffle=False,
                               collate_fn=collate)

loading dataset...



Map:   0%|          | 0/100 [00:00<?, ? examples/s]

##LLM grading

In [12]:
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch import cuda, tensor, no_grad
import torch
from tqdm.auto import tqdm
import json

device = 'cuda' if cuda.is_available() else 'cpu'
pipe = pipeline(
    "text-generation",
    model="google/gemma-2-2b-it",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    use_cache=False,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [13]:
epochs = 1

num_training_steps = epochs * len(train_dataloader)
progress_bar = tqdm(range(num_training_steps))

with open ('/content/drive/MyDrive/senior_thesis/part_2/outputs/gemma-2-2b-it_raw_pt2.csv', 'w') as out_file:
    out_file.write('id_1,id_2,prompt,pred\n')
    with no_grad():
        for _ in range(epochs):
            for _, batch in enumerate(train_dataloader):
                id_1 = batch['id_1']
                id_2 = batch['id_2']
                prompt = batch['prompt']
                # Using the text field directly for Gemma
                out = pipe(batch['text'], max_new_tokens=216, pad_token_id=pipe.tokenizer.eos_token_id)
                for i in range(len(batch['text'])):
                    # For Gemma, we'll extract the generated content differently
                    generated_text = out[i][0]["generated_text"]
                    # Extract only the model response part (after <start_of_turn>model)
                    model_response = generated_text.split("<start_of_turn>model\n")[-1].strip()
                    content = model_response.replace('"', '""')
                    out_file.write(f'{id_1[i]},{id_2[i]},{prompt[i]},"{content}"\n')
                progress_bar.update()

  0%|          | 0/13 [00:00<?, ?it/s]

You have set `use_cache` to `False`, but cache_implementation is set to hybrid. cache_implementation will have no effect.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
